In [ ]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

In [2]:
df = pd.read_csv(r'F:\study\ml\DataSet\beijinPM25\PRSA_data_2010.1.1-2014.12.31.csv')
df.head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43824 entries, 0 to 43823
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   No      43824 non-null  int64  
 1   year    43824 non-null  int64  
 2   month   43824 non-null  int64  
 3   day     43824 non-null  int64  
 4   hour    43824 non-null  int64  
 5   pm2.5   41757 non-null  float64
 6   DEWP    43824 non-null  int64  
 7   TEMP    43824 non-null  float64
 8   PRES    43824 non-null  float64
 9   cbwd    43824 non-null  object 
 10  Iws     43824 non-null  float64
 11  Is      43824 non-null  int64  
 12  Ir      43824 non-null  int64  
dtypes: float64(4), int64(8), object(1)
memory usage: 4.3+ MB


In [4]:
df['pm2.5'].isna().sum()

2067

In [5]:
df = df[24:].fillna(method='ffill')
df['pm2.5'].isna().sum()

0

In [6]:
import datetime

In [7]:
df['time']=df.apply(lambda x : datetime.datetime(year=x['year'],month=x['month'],day=x['day'],hour=x['hour']),axis=1)
df.head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,time
24,25,2010,1,2,0,129.0,-16,-4.0,1020.0,SE,1.79,0,0,2010-01-02 00:00:00
25,26,2010,1,2,1,148.0,-15,-4.0,1020.0,SE,2.68,0,0,2010-01-02 01:00:00
26,27,2010,1,2,2,159.0,-11,-5.0,1021.0,SE,3.57,0,0,2010-01-02 02:00:00
27,28,2010,1,2,3,181.0,-7,-5.0,1022.0,SE,5.36,1,0,2010-01-02 03:00:00
28,29,2010,1,2,4,138.0,-7,-5.0,1022.0,SE,6.25,2,0,2010-01-02 04:00:00


In [8]:
df.drop(columns=['year','month','day','hour','No'],inplace=True)
df=df.set_index('time')
df.head()

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
time,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0


In [9]:
df['cbwd'].unique()

array(['SE', 'cv', 'NW', 'NE'], dtype=object)

In [10]:
df=df.join(pd.get_dummies(df['cbwd']))
df.head()

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,NE,NW,SE,cv
time,,,,,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0,0,0,1,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0,0,0,1,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0,0,0,1,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0,0,0,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0,0,0,1,0


In [11]:
df.drop(columns=['cbwd'],inplace=True)
df.head()

,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,NE,NW,SE,cv
time,,,,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0


In [12]:
plt.plot(df['pm2.5'][-1000:])
plt.show()

In [13]:
df['pm2.5'][-1000:].plot()

<AxesSubplot:xlabel='time'>

In [14]:
seq_len=5*24
delay=1*24

In [15]:
df.shape

(43800, 11)

In [16]:
df_=np.array([df.iloc[i:i+seq_len+delay].values for i in range(len(df)-seq_len-delay)])
df_.shape

(43656, 144, 11)

In [17]:
np.random.shuffle(df_)
x=df_[:,:5*24,:]
y=df_[:,-1,0]
x.shape,y.shape

((43656, 120, 11), (43656,))

In [18]:
split = int(y.shape[0]*0.8)
train_x = x[:split]
train_y = y[:split]
test_x = x[split:]
test_y = y[split:]

In [19]:
mean = train_x.mean(axis=0)
std = train_x.std(axis=0)
train_x = (train_x - mean) / std
test_x = (test_x - mean) / std

In [20]:
mean.shape

(120, 11)

In [21]:
x.shape,y.shape

((43656, 120, 11), (43656,))

In [22]:
net=nn.Sequential(nn.Linear(1320,32),nn.ReLU(),nn.Linear(32,1))

In [23]:
net.state_dict

<bound method Module.state_dict of Sequential(
  (0): Linear(in_features=1320, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=1, bias=True)
)>

In [24]:
def train_k_fold_data(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.MSELoss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    train_l,test_l=[],[]
    
    min_test_loss=50000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#         if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        with torch.no_grad():
            test_loss=loss(net(test_features),test_labels).item()
            if test_loss<min_test_loss:
                min_test_loss=test_loss
#                 test_l.append(test_loss)
                train_loss=loss(net(train_features),train_labels).item()
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l

In [25]:
def get_kfold_data(k,j,x,y,random_state=13):
    assert k>=1, 'k must >=1'
    fold_size = x.shape[0] // k
    x_train,y_train=None,None
    row_list=list(range(x.shape[0]))
    np.random.seed(random_state)
    np.random.shuffle(row_list)
    for i in range(k):
        idx=slice(fold_size*i,fold_size*(i+1))
        x_part,y_part=x[row_list[idx],:],y[row_list[idx],:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val

In [26]:
def train_kfold(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')

In [27]:
def train_k_fold_data(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):

SyntaxError: unexpected EOF while parsing (<ipython-input-27-dc9ceb1fc5f2>, line 1)

In [28]:
train_x.shape

(34924, 120, 11)

In [29]:
train_features=torch.Tensor(train_x.reshape(train_x.shape[0],-1))
train_labels=torch.Tensor(train_y.reshape(train_y.shape[0],-1))
test_features=torch.Tensor(test_x.reshape(test_x.shape[0],-1))
test_label=torch.Tensor(test_y.reshape(test_y.shape[0],-1))

In [30]:
net=nn.Sequential(nn.Linear(1320,32),nn.ReLU(),nn.Linear(32,1))
train_l,test_l=train_k_fold_data(net,50,0.001,train_features,train_labels,test_features,test_label,128,0,0)

epoch = 1 train_loss : 8462.453125 , test loss : 8836.779297
epoch = 2 train_loss : 7348.428223 , test loss : 7854.933594
epoch = 3 train_loss : 6842.492188 , test loss : 7393.692871
epoch = 4 train_loss : 6561.430176 , test loss : 7173.100586
epoch = 5 train_loss : 6350.649414 , test loss : 6986.727539
epoch = 6 train_loss : 6188.131348 , test loss : 6854.463867
epoch = 7 train_loss : 6040.083984 , test loss : 6734.898438
epoch = 8 train_loss : 5922.275879 , test loss : 6653.594238
epoch = 9 train_loss : 5821.754883 , test loss : 6583.840332
epoch = 10 train_loss : 5731.559082 , test loss : 6516.039551
epoch = 11 train_loss : 5650.490723 , test loss : 6454.112793
epoch = 12 train_loss : 5548.586426 , test loss : 6380.861816
epoch = 13 train_loss : 5462.853516 , test loss : 6316.342773
epoch = 14 train_loss : 5406.032715 , test loss : 6304.373535
epoch = 15 train_loss : 5306.604004 , test loss : 6201.164551
epoch = 16 train_loss : 5254.181152 , test loss : 6197.802246
epoch = 17 train_

In [31]:
train_x.shape

(34924, 120, 11)

In [32]:
class LSTM_Reg(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,layer_size=2):
        super().__init__()
        self.lstm_net=nn.LSTM(input_size=input_size,hidden_size=hidden_size,num_layers=layer_size)
        self.dense=nn.Linear(hidden_size,output_size)
        self.state=None
    
    def forward(self,x,state):
#         x=x.reshape(x.shape[0],-1)
        y,self.state=self.lstm_net(x,state)
        output=self.dense(y.view(-1,y.shape[-1]))
        return output,self.state
        

In [33]:
net2=LSTM_Reg(1320,32,1,2)
net2.state_dict

<bound method Module.state_dict of LSTM_Reg(
  (lstm_net): LSTM(1320, 32, num_layers=2)
  (dense): Linear(in_features=32, out_features=1, bias=True)
)>

In [34]:
def train_k_fold_data_lstm(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.L1Loss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True,drop_last=True)
    train_l,test_l=[],[]
    state=None
    
    min_test_loss=50000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        test_loss_list,train_loss_list=[],[]
        net.train()
        for x,y in data_iter:
            if state is not None:
                if isinstance(state,tuple):
                    state=(state[0].detach(),state[1].detach())
                else:
                    state=state.detach()
                    
            x=x.unsqueeze(dim=0)
            (output,state)=net(x,state)
            
            l=loss(output,y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#             train_loss_list.append(l.item())
#       if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        
        with torch.no_grad():
            dataset_train=torch.utils.data.TensorDataset(train_features,train_labels)
            data_iter_train=torch.utils.data.DataLoader(dataset_train,batch_size,shuffle=True,drop_last=True)
            
            dataset_test=torch.utils.data.TensorDataset(test_features,test_labels)
            data_iter_test=torch.utils.data.DataLoader(dataset_test,batch_size,shuffle=True,drop_last=True)
            
            for xx_test,yy_test in data_iter_test:
                xx_test=xx_test.unsqueeze(dim=0)
                (output_test,state)=net(xx_test,state)
                test_loss_list.append(loss(output_test,yy_test).item())
            test_loss=np.mean(test_loss_list)

            if test_loss<min_test_loss:
                min_test_loss=test_loss
                test_l.append(test_loss)
                for xx_train,yy_train in data_iter_train:
                    xx_train=xx_train.unsqueeze(dim=0)
                    (output_train,state)=net(xx_train,state)
                    train_loss_list.append(loss(output_train,yy_train).item())

                train_loss=np.mean(train_loss_list)
                
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l

In [35]:
net2=LSTM_Reg(1320,32,1,2)
train_l,test_l=train_k_fold_data_lstm(net2,150,0.01,train_features,train_labels,test_features,test_label,128,0,0)

epoch = 1 train_loss : 65.911688 , test loss : 65.808031
epoch = 2 train_loss : 62.681681 , test loss : 63.045291
epoch = 3 train_loss : 60.095314 , test loss : 61.295246
epoch = 4 train_loss : 58.591929 , test loss : 60.065207
epoch = 5 train_loss : 57.408465 , test loss : 59.600766
epoch = 6 train_loss : 56.319662 , test loss : 58.395698
epoch = 7 train_loss : 55.872553 , test loss : 58.313080
epoch = 8 train_loss : 54.380211 , test loss : 57.664274
epoch = 9 train_loss : 53.517110 , test loss : 56.556667
epoch = 10 train_loss : 52.824518 , test loss : 55.887822
epoch = 11 train_loss : 52.445558 , test loss : 55.831364
epoch = 13 train_loss : 51.070825 , test loss : 55.131717
epoch = 14 train_loss : 50.346064 , test loss : 54.680196
epoch = 15 train_loss : 50.485292 , test loss : 54.338287
epoch = 17 train_loss : 49.130352 , test loss : 53.931227
epoch = 19 train_loss : 48.208606 , test loss : 53.292054
epoch = 20 train_loss : 47.596869 , test loss : 52.795619
epoch = 21 train_loss :

In [ ]:
class LSTM_Reg3(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,layer_size=1):
        super().__init__()
        self.lstm_net1=nn.LSTM(input_size=input_size,hidden_size=100,num_layers=layer_size,dropout=0.3)
        self.lstm_net2=nn.LSTM(input_size=input_size,hidden_size=50,num_layers=layer_size,dropout=0.2)
#         self.lstm_net3=nn.LSTM(input_size=input_size,hidden_size=50,num_layers=layer_size,dropout=0.2)
#         self.lstm_net4=nn.LSTM(input_size=input_size,hidden_size=50,num_layers=layer_size,dropout=0.2)
        self.dense1=nn.Linear(100,50)
        self.dense2=nn.Linear(50,1)
        self.h=None
        self.c=None
    
    def forward(self,x,h,c):
#         x=x.reshape(x.shape[0],-1)
        y,self.h,self.c=self.lstm_net1(x,(h,c))
        y=self.dense1(y.reshape(-1,y.shape[-1]))
        self.h=self.dense1(self.h.reshape(-1,self.h.shape[-1]))
        self.h=self.dense1(self.c.reshape(-1,self.c.shape[-1]))
        y=y.unsqueeze(dim=0)
        self.h=self.h.unsqueeze(dim=0)
        self.c=self.c.unsqueeze(dim=0)
        y,(self.h,self.c)=self.lstm_net2(y,(self.h,self.c))
        output=self.dense2(y.view(-1,y.shape[-1]))
        return output,(self.h,self.c)

In [ ]:
class LSTM_Reg4(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,layer_size=1):
        super().__init__()
        self.lstm_net1=nn.LSTM(input_size=input_size,hidden_size=100,num_layers=layer_size,dropout=0.3)
        self.lstm_net2=nn.LSTM(input_size=input_size,hidden_size=50,num_layers=layer_size,dropout=0.2)
        self.lstm_net3=nn.LSTM(input_size=input_size,hidden_size=50,num_layers=layer_size,dropout=0.2)
        self.lstm_net4=nn.LSTM(input_size=input_size,hidden_size=50,num_layers=layer_size,dropout=0.2)
        self.dense1=nn.Linear(100,50)
        self.dense2=nn.Linear(50,1)
        self.state=None
    
    def forward(self,x,state):
#         x=x.reshape(x.shape[0],-1)
        y,self.state=self.lstm_net1(x,state)
        y=self.dense1(y.reshape(-1,y.shape[-1]))
        self.state[0]=self.dense1(self.state[0].reshape(-1,self.state[0].shape[-1]))
        self.state[1]=self.dense1(self.state[1].reshape(-1,self.state[1].shape[-1]))
        y=y.unsqueeze(dim=0)
        self.state[0]=self.state[0].unsqueeze(dim=0)
        self.state[1]=self.state[1].unsqueeze(dim=0)
        y,self.state=self.lstm_net2(y,self.state)
        output=self.dense2(y.view(-1,y.shape[-1]))
        return output,self.state

In [ ]:
net1=LSTM_Reg3(1320,100,1,1)
net1.state_dict

In [ ]:
net4=LSTM_Reg3(1320,32,1,1)
train_l,test_l=train_k_fold_data_lstm(net4,150,0.01,train_features,train_labels,test_features,test_label,64,0,0)

In [36]:
class LSTM_Reg5(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,layer_size=1):
        super().__init__()
        self.lstm_net1=nn.LSTM(input_size=input_size,hidden_size=100,num_layers=layer_size,dropout=0.3)
        self.lstm_net2=nn.LSTM(input_size=100,hidden_size=50,num_layers=layer_size,dropout=0.2)
#         self.lstm_net3=nn.LSTM(input_size=input_size,hidden_size=50,num_layers=layer_size,dropout=0.2)
#         self.lstm_net4=nn.LSTM(input_size=input_size,hidden_size=50,num_layers=layer_size,dropout=0.2)
        self.dense1=nn.Linear(100,50)
        self.dense11=nn.Linear(50,100)
        self.dense2=nn.Linear(50,1)
        self.h=None
        self.c=None
    
    def forward(self,x,h,c):
#         x=x.reshape(x.shape[0],-1)if h is None:
        if h is None:
            y,(self.h,self.c)=self.lstm_net1(x,None)
        else:
            y,(self.h,self.c)=self.lstm_net1(x,(h,c))
#         y,self.h,self.c=self.lstm_net1(x,(h,c))
#         y=self.dense1(y.reshape(-1,y.shape[-1]))
        self.h=self.dense1(self.h.reshape(-1,self.h.shape[-1]))
        self.c=self.dense1(self.c.reshape(-1,self.c.shape[-1]))
#         y=y.unsqueeze(dim=0)
        self.h=self.h.unsqueeze(dim=0)
        self.c=self.c.unsqueeze(dim=0)
#         print('y.shape ',y.shape)
#         print('h.shape ',self.h.shape)
#         print('c.shape ',self.c.shape)
        y,(self.h,self.c)=self.lstm_net2(y,(self.h,self.c))
        output=self.dense2(y.view(-1,y.shape[-1]))
        self.h=self.dense11(self.h)
        self.c=self.dense11(self.c)
        return output,self.h,self.c

In [39]:
def train_k_fold_data_lstm(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.L1Loss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True,drop_last=True)
    train_l,test_l=[],[]
    h,c=None,None
    
    min_test_loss=50000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        test_loss_list,train_loss_list=[],[]
        net.train()
        print('e : ',e)
        i=1
        for x,y in data_iter:
            
            if h is not None:
                h=h.detach()
                c=c.detach()   
            x=x.unsqueeze(dim=0)
            output,h,c=net(x,h,c)
            
            l=loss(output,y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            print(i)
            i=i+1
#             train_loss_list.append(l.item())
#       if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        
        with torch.no_grad():
            dataset_train=torch.utils.data.TensorDataset(train_features,train_labels)
            data_iter_train=torch.utils.data.DataLoader(dataset_train,batch_size,shuffle=True,drop_last=True)
            
            dataset_test=torch.utils.data.TensorDataset(test_features,test_labels)
            data_iter_test=torch.utils.data.DataLoader(dataset_test,batch_size,shuffle=True,drop_last=True)
            
            for xx_test,yy_test in data_iter_test:
                xx_test=xx_test.unsqueeze(dim=0)
                (output_test,h,c)=net(xx_test,h,c)
                test_loss_list.append(loss(output_test,yy_test).item())
            test_loss=np.mean(test_loss_list)

            if test_loss<min_test_loss:
                min_test_loss=test_loss
                test_l.append(test_loss)
                for xx_train,yy_train in data_iter_train:
                    xx_train=xx_train.unsqueeze(dim=0)
                    (output_train,h,c)=net(xx_train,h,c)
                    train_loss_list.append(loss(output_train,yy_train).item())

                train_loss=np.mean(train_loss_list)
                
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l

In [40]:
net3=LSTM_Reg5(1320,32,1,1)
train_l,test_l=train_k_fold_data_lstm(net3,150,0.01,train_features,train_labels,test_features,test_label,64,0,0)

D:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
D:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


e :  0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
e :  4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
2

493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
e :  8
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
2

476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
e :  12
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205


...

283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532


262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511


238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487


218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467


199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
